## Syntax-dependent trace



In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!nvidia-smi

Thu Apr 27 11:50:27 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:18:00.0 Off |                    0 |
| N/A   39C    P0    56W / 300W |      0MiB / 32768MiB |      2%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import os
CACHE_DIR = "/scratch/lucchetti.f/models/"
os.environ['TRANSFORMERS_CACHE'] = CACHE_DIR


import sys
sys.path.append('..')

import tqdm as notebook_tqdm
# from model import ModelLoader
from model_loader import ModelLoader
from huggingface_hub import hf_hub_download
import gc
import torch

/home/lucchetti.f/.local/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def check_dev(n):
    t = torch.cuda.get_device_properties(n).total_memory
    r = torch.cuda.memory_reserved(n)
    a = torch.cuda.memory_allocated(n)
    f = r-a  # free
    print(f"{a} / {t} used for device {n}, reserved {r}")
    
def check_devs():
    for i in range(torch.cuda.device_count()):
        check_dev(i)

check_devs()

0 / 34089926656 used for device 0, reserved 0


In [5]:
MODEL_NAME = "Salesforce/codegen-16B-mono"
## Optional Download the Weights
# weights_location = hf_hub_download(MODEL_NAME, "pytorch_model.bin", cache_dir=CACHE_DIR)
# hf_hub_download(MODEL_NAME, "config.json", cache_dir=CACHE_DIR)

In [6]:
loaded_model = ModelLoader(MODEL_NAME)

Salesforce/codegen-16B-mono ==> device: cuda:0, memory: 32206917632


In [7]:
check_devs()

32206917632 / 34089926656 used for device 0, reserved 32212254720


## Inference

In [8]:
from model_utils import generate_fast

model = loaded_model.model
tokenizer = loaded_model.tokenizer
tokenizer.pad_token = tokenizer.eos_token

In [23]:
prompts = ["""# intialize three integers
           int x = 1
           int y = 2
           int z ="""]

prompts = ["""int x ="""]

# print(inputs)
# inputs = inputs.to('cpu')
# print(inputs)
# print(inputs.shape)
tokenized = tokenizer(prompts, return_tensors="pt")

outputs = model.generate(tokenized.input_ids.to(model.device), max_new_tokens=2, top_k = 1)
print(tokenizer.decode(outputs[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


int x = 0;


## Trace + activation patch